In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
amtrakHistory = pd.read_csv(r"C:\fullDataset.csv")

C:\Users\sa.DESKTOP-T984R2G\AppData\Local\Temp\ipykernel_17820\1988476898.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  amtrakHistory = pd.read_csv(r"C:\fullDataset.csv")


In [3]:
amtrakHistory = amtrakHistory.sort_values(by=['Year', 'Month', 'Day'])

In [4]:
amtrakHistory = amtrakHistory.iloc[:, :-2]

In [5]:
def convert_time(time_str):
    #This function is meant to convert every string into Military Time so that times can be compared for a machine learning algorithm
    
    if not isinstance(time_str, str):
        return np.nan
    #Our given from the dataset is that all of the times are in string form, therefore, if there is not a string, there is no time to be converted so we return null
    

    if time_str in ["*", "V", "|"]:
        return np.nan
    #This is just in existence while I was hardcoding stuff in, I decided to keep it to display that coding truly is a process and your going to constantly be adjusting
    
    if len(time_str) < 2 or time_str[-1] not in ['A', 'P']:
        return np.nan
    # This is when I realized that wait, if the string isn't at least 2 long 12A or if its last element isn't a or p it can't be a time
    
    is_pm = time_str[-1] == 'P'
    #Boolean for is PM, just add 12 to adjust for military time
   
    time_str = time_str[:-1]
    #Grabbing the numerics only in order to  convert it to the proper times

    try:
        if len(time_str) == 3:  # HMM format
            hour = int(time_str[:1])
            minute = int(time_str[1:])
        elif len(time_str) == 4:  # HM format
            hour = int(time_str[:2])
            minute = int(time_str[2:])
        else:
            return np.nan
        #Just writing all the specifications because there are multiple forms of time given in the dataset
        if not (0 <= hour < 13 and 0 <= minute < 60):
            return np.nan
    except ValueError:  
        return np.nan
    #Controlling all possible errors

    if is_pm and hour != 12:
        hour += 12
    elif not is_pm and hour == 12:  
        hour = 0
    
   #Just making sure that the time is in proper form  
    return f"{hour:02d}:{minute:02d}"


In [6]:
amtrakHistory['Schedule Arrival Time'] = amtrakHistory['Schedule Arrival Time'].apply(convert_time)
amtrakHistory['Actual Arrival Time'] = amtrakHistory['Actual Arrival Time'].apply(convert_time)
amtrakHistory['Actual Departure Time'] = amtrakHistory['Actual Departure Time'].apply(convert_time)
amtrakHistory['Schedule Departure Time'] = amtrakHistory['Schedule Departure Time'].apply(convert_time)

In [ ]:
from datetime import datetime

def compare_times(row):

    if pd.notna(row['Schedule Departure Time']) and pd.notna(row['Actual Departure Time']):
        scheduled_time = datetime.strptime(row['Schedule Departure Time'], "%H:%M").time()
        actual_time = datetime.strptime(row['Actual Departure Time'], "%H:%M").time()
   
    elif pd.notna(row['Schedule Arrival Time']) and pd.notna(row['Actual Arrival Time']):
        scheduled_time = datetime.strptime(row['Schedule Arrival Time'], "%H:%M").time()
        actual_time = datetime.strptime(row['Actual Arrival Time'], "%H:%M").time()
    
    else:
        return None

    if actual_time > scheduled_time:
        return "Late"
    elif actual_time < scheduled_time:
        return "Early"
    else:
        return "On Time"
    
amtrakHistory['Status'] = amtrakHistory.apply(compare_times, axis=1)


In [8]:
stationInfo = pd.read_csv(r"C:\Users\sa.DESKTOP-T984R2G\Downloads\amtrak.csv")

In [9]:
amtrakHistory = pd.merge(amtrakHistory, stationInfo, left_on = 'Station Code', right_on = 'STNCODE', how = 'left')

In [11]:
amtrakHistory = amtrakHistory.iloc[:, 1:]

In [12]:
amtrakHistory

,Train ID,Year,Month,Day,Station Code,Schedule Arrival Day,Schedule Arrival Time,Schedule Departure Day,Schedule Departure Time,Actual Arrival Time,...,OBJECTID,STNCODE,STNNAME,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,STNTYPE,STFIPS
0,11,2019,1,1,SEA,*,NaN,1,09:45,NaN,...,716.0,SEA,"Seattle (Amtrak), Washington",303 South Jackson Street,King St. Station,Seattle,WA,98104-2868,RAIL,53.0
1,11,2019,1,1,TAC,*,NaN,1,10:37,NaN,...,790.0,TAC,"Tacoma, Washington",1001 Puyallup Avenue,NaN,Tacoma,WA,98421-2122,RAIL,53.0
2,11,2019,1,1,OLW,*,NaN,1,11:27,NaN,...,581.0,OLW,"Olympia/Lacey, Washington",6600 Yelm Highway S.E.,NaN,Lacey,WA,98513-6481,RAIL,53.0
3,11,2019,1,1,CTL,*,NaN,1,11:51,NaN,...,51.0,CTL,"Centralia, Washington",210 Railroad Avenue,NaN,Centralia,WA,98531,RAIL,53.0
4,11,2019,1,1,KEL,*,NaN,1,12:35,NaN,...,22.0,KEL,"Kelso-Longview, Washington",501 South First Street,NaN,Kelso,WA,98626,RAIL,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4816686,99,2022,12,31,ASD,*,NaN,1,19:00,NaN,...,310.0,ASD,"Ashland, Virginia",112 North Railroad Avenue,NaN,Ashland,VA,23005,RAIL,51.0
4816687,99,2022,12,31,RVR,1,19:18,1,19:23,19:58,...,691.0,RVR,"Richmond (Staples Mill Rd), Virginia",7519 Staples Mill Road,NaN,Richmond,VA,23228,RAIL,51.0
4816688,99,2022,12,31,RVM,*,NaN,1,19:50,NaN,...,690.0,RVM,"Richmond (Main St), Virginia",1500 East Main Street,NaN,Richmond,VA,23219,RAIL,51.0
4816689,99,2022,12,31,WBG,*,NaN,1,20:43,NaN,...,479.0,WBG,"Williamsburg, Virginia",468 North Boundary Street,NaN,Williamsburg,VA,23185-3650,RAIL,51.0


In [13]:
amtrakHistory.to_csv("amtrackData.csv")